In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install pretrainedmodels

In [ ]:
from torchvision.models import *
import pretrainedmodels

from fastai import *
from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta
import fastai

from utils import *
import sys
import torch
fastai.__version__

In [ ]:
train_path = '/kaggle/input/lunar_rock/train'

In [ ]:
df = pd.read_csv('/kaggle/input/lunar_rock/train.csv')
df.head()

In [ ]:
df['Class'].value_counts()

In [ ]:
# tfms = get_transforms(flip_vert=False,max_zoom=1.0,max_warp=0,do_flip=False,xtra_tfms=[cutout()])
tfms = get_transforms(flip_vert=False,max_zoom=1.0,max_warp=0,do_flip=False)
data = (ImageList.from_csv(train_path, csv_name = '../train.csv')
        .split_by_rand_pct()              
        .label_from_df()            
        .add_test_folder(test_folder = '../test')              
        .transform(tfms, size=400)
        .databunch(num_workers=0,bs=8)) 

In [ ]:
data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(8,12))

In [ ]:
data.classes

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[error_rate, accuracy], model_dir="/tmp/model/")

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.fit_one_cycle(4, max_lr=slice(1e-6,1e-3))

In [ ]:
preds,_ = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
# learn1 = cnn_learner(data, models.resnet50, metrics=[error_rate, accuracy], model_dir='/tmp/model/')

In [ ]:
# learn1.fit_one_cycle(2)

In [ ]:
# learn1.unfreeze()
# learn1.lr_find()
# learn1.fit_one_cycle(2, max_lr=slice(1e-6, 1e-3))

In [ ]:
# preds1,_ = learn1.TTA(ds_type=DatasetType.Test)

In [ ]:
labelled_preds = []
ensemble_preds = preds
for pred in ensemble_preds:
    labelled_preds.append(int(np.argmax(pred))+1)


In [ ]:
actual_labelled_preds = []
for i in labelled_preds:
    if i == 1:
        actual_labelled_preds.append('Large')
    else:
        actual_labelled_preds.append('Small')
    
submission = pd.DataFrame(
    {'Image_File': filenames,
     'Class': actual_labelled_preds,
    })
submission.to_csv('submission.csv',index=False)

In [ ]:
submission.head()

In [ ]:
submission = submission.sort_values(by = ['Image_File'], ascending = [True])
submission

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "resnet34-4_cycle.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission)